In [1]:
#!pip install nltk
#!python -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import re # -> Regular Expression (Very useful for searching the text in the document)
from nltk.corpus import stopwords # -> natural language toolkit
from nltk.stem.porter import PorterStemmer # -> takes a word and removes the suffix and prefix and return the Root of that word
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ahmed\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# -> Printing StopWords in English
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

# Data Pre-Processing

In [5]:
news_data =  pd.read_csv("train.csv")

In [6]:
news_data.head()
# -> 0 Real News 
# -> Fake News

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
news_data.shape

(20800, 5)

In [8]:
# -> Counting the number of the missing values in the dataset in each column
news_data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [9]:
# -> Replacing the null values with an empty string 
news_data = news_data.fillna('')

In [16]:
# -> merging the author's name and news title
news_data['Content'] = news_data['author']+' '+news_data['title'] 

In [11]:
print(news_data['Content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: Content, Length: 20800, dtype: object


In [12]:
# -> Seperating label & Data 
X = news_data.drop(columns='label', axis =1)
Y = news_data['label']

In [13]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

**Stemming**
-> is the process of reducing a word to its root word (to have a better performance in our model)
**Example** -> actress , actor , acting --> act

In [14]:
port_stem = PorterStemmer()

In [15]:
# -> def =====> defined 
def stemming(Content):
    # ==> (^) --> for exclusion ===> this for removing everything not alphabets --> anything not an alphabet will be replaced by space
    stemmed_content = re.sub('[^a-zA-Z]',' ', Content) 
    stemmed_content = stemmed_content.lower() # --> convert all of the alphabets to lower case
    stemmed_content = stemmed_content.split() # --> convert all words to a list
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    # --> This for stemmed all words and removing all StopWords
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [17]:
news_data['Content'] = news_data['Content'].apply(stemming)

In [18]:
print(news_data['Content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: Content, Length: 20800, dtype: object


In [19]:
# -> Seperating the data and label 
X = news_data['Content'].values
Y = news_data['label'].values

In [21]:
print(X)
print(Y)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']
[1 0 1 ... 0 1 1]


In [23]:
Y.shape

(20800,)

In [30]:
# --> Converting the textual data to numerical data
# ====> Counts the number of words is repeating in a document 
vectorizer = TfidfVectorizer() # -> stands for ==> term frequency inverse document frequency
X = vectorizer.fit_transform(X)

AttributeError: 'csr_matrix' object has no attribute 'lower'

In [31]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 210687 stored elements and shape (20800, 17128)>
  Coords	Values
  (0, 3600)	0.3598939188262559
  (0, 8909)	0.3635963806326075
  (0, 7005)	0.21874169089359144
  (0, 3792)	0.2705332480845492
  (0, 267)	0.27010124977708766
  (0, 4973)	0.233316966909351
  (0, 13473)	0.2565896679337957
  (0, 2959)	0.2468450128533713
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 2483)	0.3676519686797209
  (0, 15686)	0.28485063562728646
  (1, 3568)	0.26373768806048464
  (1, 5503)	0.7143299355715573
  (1, 6816)	0.1904660198296849
  (1, 2813)	0.19094574062359204
  (1, 1497)	0.2939891562094648
  (1, 16799)	0.30071745655510157
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (2, 3103)	0.46097489583229645
  (2, 2943)	0.3179886800654691
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  :	:
  (20797, 9588)	0.17455348025522197
  (20797, 7042)	0.21799048897828685
  (207

# Spliting the dataset into training and test data

In [37]:
X_train , X_test , Y_train , Y_test = train_test_split( X , Y , test_size =0.2 ,stratify = Y , random_state = 2)
# -> When you do not mention stratify = Y label's content won't be segregated in equal proportion
# ==> random_state = 2 The data will be splitting the same way each time (can be any integer value)

# Training The Model: Logistic Regression

In [38]:
model = LogisticRegression()

In [39]:
model.fit(X_train ,Y_train)

LogisticRegression()

# Evaluation

In [41]:
x_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(x_train_prediction , Y_train)
print ("Accuracy score on the training data:",training_data_accuracy)

Accuracy score on the training data: 0.9863581730769231


In [42]:
x_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(x_test_prediction , Y_test)
print ("Accuracy score on the test data:",test_data_accuracy)

Accuracy score on the test data: 0.9790865384615385


# Making a predictive system

In [57]:
X_new = X_test[2] # ---> means the first row

prediction = model.predict(X_new)
print(prediction)

[1]


In [58]:
if (prediction[0] ==  1):
    print("Fake News")
else :
    print("Real News")

Fake News


In [59]:
print(Y_test[2])

1
